# Oppgave 3
Her skal du kjøpe 9 voksenbilletter til forestillingen for Størst av alt er
kjærligheten 3. februar, hvor det er 9 ledige billetter og hvor stolene er på
samme rad. Stolene trenger ikke være ved siden av hverandre. Vi ønsker å få
summert hva det koster å kjøpe disse billettene,

In [3]:
import sqlite3
import pandas as pd
from tabulate import tabulate
import os

def get_file_name_of_newest_database():
    base_name = "database"
    counter = 1

    # Increment counter until a new file name is found
    while os.path.exists(f"{base_name}{counter}.db"):
        counter += 1
    counter -= 1
    return f"{base_name}{counter}.db"

name = get_file_name_of_newest_database()
conn = sqlite3.connect(name)
cursor = conn.cursor()

In [7]:
cursor.execute("SELECT * FROM Kundeprofil")
customer = cursor.fetchone()
customer

date = "2024-02-03"

query = "SELECT Nummer FROM Forestilling WHERE teaterstykkeNavn = 'Størst av alt er kjærligheten' AND dato = ?"
cursor.execute(query, (date,))
ForestillingNummer = cursor.fetchone()[0]

# billettkjøp
cursor.execute(
    "INSERT INTO Billettkjøp (TeaterStykkeNavn,ForestillingNummer, Nummer, Mobilnummer, dato, tid) VALUES (?, ?,?,?, ?, ?)",
    (
        "Størst av alt er kjærligheten",
        ForestillingNummer,
        2,
        "1234567890",
        date,
        "18:30",
    ),
)
conn.commit()

# billett
plasser = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# kjøp 9 plasser rad 1  parkett
for plass in plasser:
    cursor.execute(
        "INSERT INTO Billett (RadNr, StolNr, Område, salNavn, ForestillingNummer, BillettKjøpNummer, Billettype, TeaterStykkeNavn ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
        (
            2,
            plass,
            "Galleri",
            "Gamle scene",
            ForestillingNummer,
            2,
            "Ordinær",
            "Størst av alt er kjærligheten",
        ),
    )
conn.commit()

IntegrityError: UNIQUE constraint failed: Billettkjøp.Nummer, Billettkjøp.ForestillingNummer, Billettkjøp.TeaterStykkeNavn

In [5]:
# get ticket for forestilling 6 and billettkjøp 2 and join them with ticket type and sum the price

query = """
SELECT SUM(BilletType.pris)
FROM Billett
JOIN BilletType
ON Billett.Billettype = BilletType.type
AND Billett.TeaterStykkeNavn = BilletType.TeaterStykkeNavn
WHERE Billett.ForestillingNummer = 6
AND Billett.BillettKjøpNummer = 2
"""
cursor.execute(query)
tickets = cursor.fetchall()
print(f'Billettene koster {tickets[0][0]} kr')

Billettene koster None kr
